In [ ]:
import warnings
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['M5','M15','H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    
    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time
    Src['ID T']                         = Src.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)['Time'].rank()

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    Src['Change']       = (Src['Close'] - Src['Open'])
    Src['Hilo']         = (Src['High']  - Src['Low'])

    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if x >= 0 else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if x <  0 else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

# Stats

In [ ]:
Df1 = Src[Src['ID T']==1] .reset_index(drop=1)

Df2 = Df1.sort_values(['Date','TF'], ascending=[True,False])  .reset_index(drop=1)
Df2['Count'] = Df2.groupby('Date', sort=0, group_keys=0)['Symbol'].transform('count')

Df3 = Df2[Df2['Count']==3] .reset_index(drop=1)

In [94]:
(Df3.groupby('Date', sort=0, group_keys=0)['Sign Bull'].sum().eq(+3).sum() / len(Df3['Date'].unique())) *100

np.float64(31.38121546961326)

In [93]:
(Df3.groupby('Date', sort=0, group_keys=0)['Sign Bear'].sum().eq(-3).sum() / len(Df3['Date'].unique())) *100

np.float64(31.8232044198895)

# Snippets